In [1]:
from ContentUtil import ContentUtil
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

Load data from folders.

In [2]:
ctl = ContentUtil()
AI_data, Not_AI_data = ctl.load_with_label("datasets/AI/", "datasets/NOT/")
AI = np.array(AI_data)
Not_AI = np.array(Not_AI_data)
data = np.concatenate((AI, Not_AI), axis=0)

In [3]:
df = pd.DataFrame(data,columns=["data","label"])
df.head()

,data,label
0,Amerucan computer gaming and artificil intelli...,AI
1,American political scientist economist sociolo...,AI
2,Meta algorithmic technique to choose an algori...,AI
3,science fiction trilogy by Peter F Hamilton T...,AI
4,The Istituto italiano per l Intelligenza artif...,AI


In [4]:
dfshf = shuffle(df).reset_index(drop=True)
dfshf.head()

,data,label
0,This article contains content that is written ...,AI
1,Research institute of the University of Southe...,AI
2,Visual arts in Sri Lanka refers to a variety o...,Art
3,Hands on computing is a branch of human comput...,AI
4,American legal case This article includes a li...,Art


In [5]:
df["label"].value_counts().plot(kind='bar',title="Distribution of Dataset")

In [8]:
import pandas as pd
import numpy as np
import string
import re
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [73]:
from tensorflow.keras.layers import LSTM, Activation, Dropout, Dense, Input,Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
from tensorflow.keras import layers

Transform text label into one-hot encoding labels

In [41]:
y_d = df["label"].to_numpy().reshape(-1,1)
y_label = OneHotEncoder().fit(y_d).transform(y_d).toarray()

Add a column for text data length

In [70]:
df["length"] = df["data"].apply(lambda x:len(x.split()))

Split data and label to train set and test set

In [43]:
X = df["data"].to_numpy()

In [44]:
X_train, X_test,Y_train, Y_test = train_test_split(X, y_label, test_size=0.2, random_state = 2021)

Define a tokenizer from Keras package and generate a dictionary for word and index. Use this dictionary to fill out the Glove embedding vector matrix.

In [45]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
words_to_index = tokenizer.word_index

Read the glove vector from files

In [48]:
def read_glove_vector(glove_vec):
    with open(glove_vec, 'r', encoding='UTF-8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            w_line = line.split()
            curr_word = w_line[0]
            word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)
    return word_to_vec_map

In [49]:
word_to_vec_map = read_glove_vector('glove.6B.100d.txt')

In [100]:
word_to_vec_map["the"]

array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
       -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
        0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
       -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,
        0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,
       -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,
        0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,
        0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,
       -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,
       -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,
       -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,
       -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,
       -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,
       -1.2526  ,  0.071624,  0.70565 ,  0.49744 , 

Define and initiate the embedding matrix with the shape (vocabulary_length, embedding length)

In [56]:
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['the'].shape[0]
print("voc_length is {} and embedding length is {}".format(vocab_len,embed_vector_len))
emb_matrix = np.zeros((vocab_len, embed_vector_len))
print("embedding matrix shape is {}".format(emb_matrix.shape))

voc_length is 149860 and embedding length is 100
embedding matrix shape is (149860, 100)


In [69]:
for word, index in words_to_index.items():
    embedding_vector = word_to_vec_map.get(word)
    if embedding_vector is not None:
        emb_matrix[index-1, :] = embedding_vector

This model contains two Bi-directional LSTM layers together with two Dropout layers. Coming with three Dense layers. As for classifier layer, use sigmoid as activation function.

In [83]:
embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=200, weights = [emb_matrix], trainable=False)


In [85]:
X_indices = Input(shape=(None,))
embeddings = embedding_layer(X_indices)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(embeddings)
x = Dropout(0.4)(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
x = Dropout(0.4)(x)
# Add a classifier
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)
outputs = layers.Dense(3, activation="sigmoid")(x)
model = keras.Model(inputs=X_indices, outputs=outputs)

In [86]:
model.summary()


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 100)         14986000  
_________________________________________________________________
bidirectional_4 (Bidirection (None, None, 128)         84480     
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 128)         0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 128)               98816     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               1651

In [87]:
X_train_indices = tokenizer.texts_to_sequences(X_train)
X_train_indices = pad_sequences(X_train_indices, maxlen=200, padding='post')

In [90]:
X_test_indices = tokenizer.texts_to_sequences(X_test)
X_test_indices = pad_sequences(X_test_indices, maxlen=200, padding='post')

In [92]:
rmsp = keras.optimizers.RMSprop(learning_rate = 0.01)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

Validating with test dataset

In [93]:
history = model.fit(X_train_indices, Y_train, batch_size=128, epochs=20,validation_data=(X_test_indices, Y_test))

Train on 3282 samples, validate on 821 samples
Epoch 1/20
3282/3282 [==============================] - 17s 5ms/sample - loss: 0.9332 - acc: 0.5771 - val_loss: 0.9236 - val_acc: 0.5688
Epoch 2/20
3282/3282 [==============================] - 16s 5ms/sample - loss: 0.8740 - acc: 0.5957 - val_loss: 0.8874 - val_acc: 0.6188
Epoch 3/20
3282/3282 [==============================] - 16s 5ms/sample - loss: 0.8448 - acc: 0.6106 - val_loss: 0.8148 - val_acc: 0.6407
Epoch 4/20
3282/3282 [==============================] - 15s 5ms/sample - loss: 0.7405 - acc: 0.6752 - val_loss: 0.7113 - val_acc: 0.6906
Epoch 5/20
3282/3282 [==============================] - 17s 5ms/sample - loss: 0.6594 - acc: 0.7246 - val_loss: 0.8476 - val_acc: 0.6151
Epoch 6/20
3282/3282 [==============================] - 16s 5ms/sample - loss: 0.6915 - acc: 0.7118 - val_loss: 0.6957 - val_acc: 0.7052
Epoch 7/20
3282/3282 [==============================] - 17s 5ms/sample - loss: 0.6034 - acc: 0.7562 - val_loss: 0.6330 - val_acc: 0

In [94]:
history2 = model.fit(X_train_indices, Y_train, batch_size=128, epochs=20,validation_data=(X_test_indices, Y_test))

Train on 3282 samples, validate on 821 samples
Epoch 1/20
3282/3282 [==============================] - 15s 4ms/sample - loss: 0.1864 - acc: 0.9342 - val_loss: 0.6700 - val_acc: 0.7881
Epoch 2/20
3282/3282 [==============================] - 15s 5ms/sample - loss: 0.1729 - acc: 0.9415 - val_loss: 0.7595 - val_acc: 0.7990
Epoch 3/20
3282/3282 [==============================] - 15s 5ms/sample - loss: 0.1403 - acc: 0.9525 - val_loss: 0.7784 - val_acc: 0.7881
Epoch 4/20
3282/3282 [==============================] - 15s 5ms/sample - loss: 0.1342 - acc: 0.9546 - val_loss: 0.7570 - val_acc: 0.8051
Epoch 5/20
3282/3282 [==============================] - 15s 5ms/sample - loss: 0.1232 - acc: 0.9592 - val_loss: 1.0434 - val_acc: 0.7333
Epoch 6/20
3282/3282 [==============================] - 15s 5ms/sample - loss: 0.1486 - acc: 0.9458 - val_loss: 0.8110 - val_acc: 0.7978
Epoch 7/20
3282/3282 [==============================] - 15s 5ms/sample - loss: 0.1044 - acc: 0.9662 - val_loss: 0.9006 - val_acc: 0

In [97]:
model.save("./result/Bi-LSTM_model")